# Ex 2: Manipulating Spectra Part 2 - `Spectra`

## Notebook Initialization

In [1]:
%load_ext autoreload
import sys
sys.path.append("..")

In [2]:
%autoreload
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize'] = [10,10]

import numpy as np

import darkhistory.physics as physics
import darkhistory.spec.spectools as spectools
from darkhistory.spec.spectrum import Spectrum
from darkhistory.spec.spectra import Spectra

## `Spectra` Class - Introduction

The `Spectra` class is simply a collection of `Spectrum` objects. The main use of `Spectra` is as a list of spectra produced by the same process at different redshifts. It is the parent class of `TransFuncAtEnergy` and `TransFuncAtRedshift`, which are useful classes for transfer functions described in a later example. 

To demonstrate how to use this class, consider the photon spectrum produced by a massive dark matter particle $\phi$ decaying into an electron pair from a two-body decay + FSR process, given approximately by (see Essig et al. 1309.4091):

$$ \frac{dN_\gamma}{dE}_{\phi \to e^+ e^- \gamma} \simeq \frac{2\alpha_{\text{EM}}}{\pi E_\gamma} \left[1 - \frac{2E_\gamma}{m_\phi} + \left(1 - \frac{2E_\gamma}{m_\phi} + \frac{2E_\gamma^2}{m_\phi^2} \right) \log \left( \frac{1 - 2E_\gamma/m_\phi}{m_e^2/m_\phi^2} \right) \right] $$

For simplicity, we consider this process during the cosmic dark ages, where the dark matter density was uniform, and that the branching ratio into this process is 100%. We will also not concern ourselves with the overall normalization of this spectrum. 

We will again use `spec.spectools.discretize` to create `Spectrum` objects from this analytic spectrum, taking $m_\phi = $ 2 TeV.

In [ ]:
def phi_decay_FSR(eng):
    m_phi = 2000
    m_e = 0.511e-3
    r = eng/m_phi
    r_e = m_e/m_phi
    return (
        (2/137)/(np.pi*eng)*(
            1 - 2*r + (1 - 2*r + 2*r**2)
        )*np.log((1 - 2*r)/r_e**2)
    )
    